# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [3]:
# Load the file with .env extension
%load_ext dotenv
%dotenv


In [4]:
import dask.dataframe as dd

c:\Users\syeolekar\.conda\envs\dsi_participant\lib\site-packages\dask\dataframe\_pyarrow_compat.py:15: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [38]:
import os
from glob import glob

# Load environment variable price data 
pd_path = os.getenv("PRICE_DATA")
join_parquet = glob(os.path.join(pd_path, "**/*.parquet"), recursive = True)
read_parquet = dd.read_parquet(join_parquet).set_index("Ticker")
read_parquet



,Date,Adj Close,Close,High,Low,Open,Volume,Year
npartitions=13078,,,,,,,,
,"datetime64[ns, UTC]",float64,float64,float64,float64,float64,float64,int32
,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [15]:
# Add lags for variable Close
pf_closelag = read_parquet.groupby("Ticker", group_keys=False).apply(
    lambda x: x.assign(close_lag_1 = x["Close"].shift(1))
)
pf_closelag


C:\Users\syeolekar\AppData\Local\Temp\ipykernel_8400\2870455947.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  pf_closelag = read_parquet.groupby("Ticker", group_keys=False).apply(


,Date,Adj Close,Close,High,Low,Open,Volume,Year,close_lag_1
npartitions=13078,,,,,,,,,
,"datetime64[ns, UTC]",float64,float64,float64,float64,float64,float64,int32,float64
,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...


In [19]:
# Add lag for variable Adj_Close
pf_adjcloselag = pf_closelag.groupby("Ticker", group_keys=False).apply(
    lambda x: x.assign(adj_close_lag_1 = x["Adj Close"].shift(1))
)
pf_adjcloselag

C:\Users\syeolekar\AppData\Local\Temp\ipykernel_8400\668032219.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  pf_adjcloselag = pf_closelag.groupby("Ticker", group_keys=False).apply(


,Date,Adj Close,Close,High,Low,Open,Volume,Year,close_lag_1,adj_close_lag_1
npartitions=13078,,,,,,,,,,
,"datetime64[ns, UTC]",float64,float64,float64,float64,float64,float64,int32,float64,float64
,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...


In [21]:
# Add returns
pf_returns = pf_adjcloselag.assign(
    returns = lambda x: x["Close"]/x["close_lag_1"]-1)
pf_returns

,Date,Adj Close,Close,High,Low,Open,Volume,Year,close_lag_1,adj_close_lag_1,returns
npartitions=13078,,,,,,,,,,,
,"datetime64[ns, UTC]",float64,float64,float64,float64,float64,float64,int32,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...


In [41]:
#add hi low range
dd_feat = pf_returns.assign(
    hi_low = lambda x: x["High"] - x["Low"]
    )
dd_feat

,Date,Adj Close,Close,High,Low,Open,Volume,Year,close_lag_1,adj_close_lag_1,returns,hi_low
npartitions=13078,,,,,,,,,,,,
,"datetime64[ns, UTC]",float64,float64,float64,float64,float64,float64,int32,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [45]:
filtered_dd_feat = dd_feat.loc[:, ["Date", "returns"]]
filtered_dd_feat

,Date,returns
npartitions=13078,,
,"datetime64[ns, UTC]",float64
,...,...
...,...,...
,...,...
,...,...


In [48]:
# Convert dask to pandas 
filtered_dd_feat.compute().sort_values(["Ticker", "Date"], ascending=[True, True])


Price,Date,returns
Ticker,,
A,2000-01-03 00:00:00+00:00,3.008909
A,2000-01-04 00:00:00+00:00,-0.076389
A,2000-01-05 00:00:00+00:00,-0.062030
A,2000-01-06 00:00:00+00:00,-0.038076
A,2000-01-07 00:00:00+00:00,0.083333
...,...,...
ZTS,2025-01-24 00:00:00+00:00,0.009883
ZTS,2025-01-27 00:00:00+00:00,0.026214
ZTS,2025-01-28 00:00:00+00:00,-0.015084


In [52]:
# new features called rolling average of returns 
pd_final = filtered_dd_feat.groupby("Ticker", group_keys=False).apply(
    lambda x: x.assign(rolling_average = x["returns"].rolling(10).mean())
)
pd_final

C:\Users\syeolekar\AppData\Local\Temp\ipykernel_8400\535227823.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  pd_final = filtered_dd_feat.groupby("Ticker", group_keys=False).apply(


,Date,returns,rolling_average
npartitions=13078,,,
,"datetime64[ns, UTC]",float64,float64
,...,...,...
...,...,...,...
,...,...,...
,...,...,...


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

Dask is a lazy execution framework. To compute moving average return, we had to compute return. When we compute, dask returns results as pandas dataframe. 

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.